## Tasks
1. Figure out how to connect widgets to the actual PycoreSpec. 
2. What are some default values? ✅
3. Debugging with Metadata? I think they have something pre-build for error check. 🤔
4. Visualizing with simulationers

In [1]:
# Imports 
import os
import json
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
from IPython.display import display
from cc3d.core.PyCoreSpecs import Metadata, PottsCore, CellTypePlugin

# from cc3d import CompuCellSetup

In [ ]:
# Data Persistency 
SAVE_FILE = 'simutation_setup.json'

# Default values
defaults = {
    "Metadata": {
        "num_processors": 4,
        "debug_output_frequency": 10
    },
    "PottsCore": {
        "dim_x": 101,
        "dim_y": 101,
        "dim_z": 1,
        "steps": 100000,
        "anneal": 0,
        "fluctuation_amplitude": 10.0,
        "fluctuation_amplitude_function": "Min",
        "boundary_x": "NoFlux",
        "boundary_y": "NoFlux",
        "boundary_z": "NoFlux",
        "neighbor_order": 1,
        "random_seed": None,
        "lattice_type": "Cartesian",
        "offset": 0
    },
    "CellType": [
        {"Cell type": "Medium", "freeze": False}
    ]
}

# Load saved values or use defaults
if os.path.exists(SAVE_FILE):
    with open(SAVE_FILE, 'r') as f:
        saved_values = json.load(f)
else:
    saved_values = defaults.copy()

def save_to_json(*args):
    with open(SAVE_FILE, 'w') as f:
        json.dump(current_config, f, indent=4)

def current_config():
    return {
        "Metadata": {
            "num_processors": num_processors.value,
            "debug_output_frequency": debug_output_frequency.value
        },
        "PottsCore": {
            "dim_x": dim_x.value + 1, # Adjusting for store values
            "dim_y": dim_y.value + 1,
            "dim_z": dim_z.value + 1,
            "steps": steps.value,
            "anneal": anneal.value,
            "fluctuation_amplitude": fluctuation_amplitude.value,
            "fluctuation_amplitude_function": fluctuation_amplitude_function.value,
            "boundary_x": boundary_x.value,
            "boundary_y": boundary_y.value,
            "boundary_z": boundary_z.value,
            "neighbor_order": neighbor_order.value,
            "random_seed": random_seed.value if random_seed.value else None,
            "lattice_type": lattice_type.value,
            "offset": offset.value
        },
        "CellType": {
            "Type": cell_type.value,
            "Freeze": freeze_cell_type.value
        }
    }

# Widgets

In [ ]:

# Metadata
num_proc = widgets.IntText(
    value=saved_values["Metadata"]["num_processors"],
    description='Number of Processors:',
    style={'description_width': 'initial'}
)

debug_freq = widgets.IntText(
    value=saved_values["Metadata"]["debug_output_frequency"],
    description='Debug Output Frequency:'
    style={'description_width': 'initial'},
)

# PottsCore 
x_slider = widgets.IntSlider(
    value=saved_values["PottsCore"]["dim_x"] - 1,  min=0, max=100,
    description='X:'
)

y_slider = widgets.IntSlider(
    value=saved_values["PottsCore"]["dim_y"] - 1,  min=0, max=100,
    description='Y:'
)

z_slider = widgets.IntSlider(
    value=saved_values["PottsCore"]["dim_z"] - 1,  min= 0, max=100,
    description='Z:'
)

steps_input = widgets.IntText(
    value=saved_values["PottsCore"]["steps"],
    description='MC Steps:'
)

anneal_input = widgets.FloatText(
    value=saved_values["PottsCore"]["anneal"],
    description='Anneal:'
)

flunctuation_slider = widgets.FloatSlider(
    value=saved_values["PottsCore"]["fluctuation_amplitude"],
    min=0.0, max=50.0, step=0.1, # what's the max value?
    description='Fluctuation:'
)

flunct_fn_dropdown = widgets.Dropdown(
    options=['Min', 'Max', 'ArithmeticAverage'],
    value=saved_values["PottsCore"]["fluctuation_amplitude_function"],
    description='Fluctuation Function:'
)

boundary_x = widgets.Dropdown(
    options=['NoFlux', 'Periodic'],
    value=saved_values["PottsCore"]["boundary_x"],
    description='Boundary X:'
)

boundary_y = widgets.Dropdown(
    options=['NoFlux', 'Periodic'],
    value=saved_values["PottsCore"]["boundary_y"],
    description='Boundary Y:'
)

boundary_z = widgets.Dropdown(
    options=['NoFlux', 'Periodic'],
    value=saved_values["PottsCore"]["boundary_z"],
    description='Boundary Z:'
)

neighbor_order_input = widgets.BoundedIntText(
    value=saved_values["PottsCore"]["neighbor_order"], min=1, max=4, # max?
    description='Neighbor Order:'
)

seed_input = widgets.Text(
    value='' if saved_values["PottsCore"]["random_seed"] is None else str(saved_values["PottsCore"]["random_seed"]),
    description='Random Seed:'
)

lattice_dropdown = widgets.Dropdown(
    options=['Square', 'Hexagonal'],
    value='Square' if saved_values["PottsCore"]["lattice_type"] == 'Cartesian' else 'Hexagonal',
    description='Lattice Type:'
)

offset_input = widgets.IntText(
    value=saved_values["PottsCore"]["offset"],
    description='Offset:'
)

In [10]:
# Cell Type

preset_dropdown = widgets.Dropdown(
    options=["Condensing", "NonCondensing", "Customize"],
    value="Condensing",
    description="Cell Type:",
    layout=Layout(width='250px', min_width='250px')
)

freeze_checkbox = widgets.Checkbox(
    value=True,
    description="Freeze",
    layout=Layout(margin='0 0 0 12px', width='120px'),
    style={'description_width': 'initial'}
)

custom_name_input = widgets.Text(
    description="Name:",
    placeholder="e.g. T1",
    layout=Layout(width='250px', min_width='250px')
)

add_button = widgets.Button(
    description="Add",
    button_style="success",
    layout=Layout(margin='10px 0 0 0', width='80px')
)

celltype_display_box = widgets.VBox(
    layout=Layout(
        border='1px solid gray',
        padding='10px',
        width='300px',
        height='auto',
        overflow_y='auto'
    )
)

def toggle_custom_input(change):
    custom_name_input.layout.display = 'block' if change['new'] == "Customize" else 'none'

preset_dropdown.observe(toggle_custom_input, names='value')
toggle_custom_input({'new': preset_dropdown.value})

def update_celltype_display():
    items = []
    for i, entry in enumerate(celltype_entries):
        label_str = entry["Cell type"]
        if entry["freeze"]:
            label_str += " (frozen)"
        label = widgets.Label(label_str, layout=Layout(flex='1 1 auto'))
        remove_btn = widgets.Button(description="Remove", button_style='danger', layout=Layout(width='80px'))
        def remove_closure(index=i):
            def remove_clicked(b):
                del celltype_entries[index]
                update_celltype_display()
            return remove_clicked
        remove_btn.on_click(remove_closure())
        item_row = HBox([label, remove_btn], layout=Layout(justify_content='space-between'))
        items.append(item_row)
    celltype_display_box.children = items

def on_add_clicked(b):
    selected = preset_dropdown.value
    if selected == "Customize":
        name = custom_name_input.value.strip()
        if not name:
            custom_name_input.placeholder = "Please enter a name!"
            return
    else:
        name = selected
    freeze = freeze_checkbox.value
    celltype_entries.append({"Cell type": name, "freeze": freeze})
    update_celltype_display()
    custom_name_input.value = ""
    freeze_checkbox.value = True

add_button.on_click(on_add_clicked)

custom_name_input.layout.display = 'none'

# Labels for columns (like form labels)
label_current_types = widgets.Label("Current Cell Types", layout=Layout(padding='0 0 8px 0'))
label_add_type = widgets.Label("Add Cell Type", layout=Layout(padding='0 0 8px 0'))

left_column = VBox([
    label_current_types,
    celltype_display_box
], layout=Layout(width='300px', padding='0 40px 0 0'))

right_column = VBox([
    label_add_type,
    preset_dropdown,
    custom_name_input,
    freeze_checkbox,
    add_button
], layout=Layout(width='300px', padding='0 0 0 40px'))

celltype_section = VBox([
    HBox([left_column, right_column], layout=Layout(justify_content='flex-start', gap='40px'))
])

TypeError: list indices must be integers or slices, not str

In [ ]:
# Reset Button
reset_button = widgets.Button(description="Reset to Defaults", button_style='warning')
def reset_to_defaults(btn):
    for key, val in defaults["Metadata"].items():
        if key == "num_processors":
            num_proc.value = val
        elif key == "debug_output_frequency":
            debug_output_frequency.value = val
    for key, val in defaults["PottsCore"].items():
        if key == "dim_x":
            dim_x.value = val - 1  # Adjusting for store values
        elif key == "dim_y":
            dim_y.value = val - 1
        elif key == "dim_z":
            dim_z.value = val - 1
        elif key == "steps":
            steps.value = val
        elif key == "anneal":
            anneal.value = val
        elif key == "fluctuation_amplitude":
            fluctuation_amplitude.value = val
        elif key == "fluctuation_amplitude_function":
            fluctuation_amplitude_function.value = val
        elif key == "boundary_x":
            boundary_x.value = val
        elif key == "boundary_y":
            boundary_y.value = val
        elif key == "boundary_z":
            boundary_z.value = val
        elif key == "neighbor_order":
            neighbor_order.value = val
        elif key == "random_seed":
            seed_input.value = '' if val is None else str(val)
        elif key == "lattice_type":
            lattice_dropdown.value = 'Square' if val == 'Cartesian' else 'Hexagonal'
        elif key == "offset":
            offset.value = val
    cell_type_radio.value = defaults["CellType"]["Type"]
    freeze_cell_type_checkbox.value = defaults["CellType"]["Freeze"]
    save_to_json()

reset_button.on_click(reset_to_defaults)

# Auto save on change
for widget in [
    num_proc, debug_freq,
    x_slider, y_slider, z_slider,
    steps_input, anneal_input,
    flunctuation_slider, flunct_fn_dropdown,
    boundary_x, boundary_y, boundary_z,
    neighbor_order_input, seed_input,
    lattice_dropdown, offset_input,
    cell_type_radio, freeze_cell_type_checkbox
]:
    widget.observe(save_to_json, 'value')

NameError: name 'dim_x' is not defined

NameError: name 'dim_x' is not defined

NameError: name 'dim_x' is not defined

NameError: name 'dim_x' is not defined

NameError: name 'dim_x' is not defined

NameError: name 'dim_x' is not defined

NameError: name 'dim_x' is not defined

In [ ]:
# Layout and display
tab_0 = VBox([widgets.HTML("<h4>Basic Simulation Setup</h4>"), num_proc, debug_freq])
tab_1 = VBox([widgets.HTML("<h4>PottsCore Parameters</h4>"),
              x_slider, y_slider, z_slider,
              steps_input, anneal_input,
              flunctuation_slider, flunct_fn_dropdown,
              HBox([boundary_x, boundary_y, boundary_z,]),
              neighbor_order_input, seed_input,
              lattice_dropdown, offset_input])
tab_2 = VBox([widgets.HTML("<h4>Cell Type Parameters</h4>"),
              cell_type_radio, freeze_cell_type_checkbox])

tabs = widgets.Tab(children=[tab_0, tab_1, tab_2])
tabs.set_title(0, 'Meta data')
tabs.set_title(1, 'PottsCore')
tabs.set_title(2, 'Cell Type')

In [ ]:
display(VBox([tabs, reset_button]))
# Values can't be negative solve by set min=0 
# Default button does not work because of name error 
# Cell type format & freeze location is off, the cell type is probably a multi add, not all cell need to be. 

# Basic Setup
1. What are the parameters? What can be default and what can be customized? ✅

## Metadata
Threads & debug output: 
1. ? number of CPU/Processor used for simulation ```num_processors```
2. ? frequncy for debugging ```debug_output_frequency```

Default values:

```num_processors = 4```

```debug_output_frequency = 10```

## PottsCore
All parameters for Cellular Potts Model:
1. Lattice dimensions: ```dim_x, dim_y, dim_z```
2. Number of Monte Carlos Steps: ```steps```
3. Anneal (Number of annealing steps/initial relaxation): ```anneal```
4. Amplitude of pixel copy fluctuations (temp.): ```fluctuation_amplitude```
5. Fluctuation calculation: ```fluctuation_amplitude_function```
6. Boundary conditions: ```boundary_x, boundary_y, boundary_z```
7. Neighborhood order: ```neighbor_order```
8. Random seed: ```random_seed```
9. Lattice geometry: ```lattice_type```
10. Lattice offset (for hexagonal lattices): ```offset ``` 

Default values:

```dim_x, dim_y, dim_z = 1,1,1```

```steps = 0```

```anneal = 0```

```fluctuation_amplitude = 10.0```

```fluctuation_amplitude_function = Min```

```boundary_x, boundary_y, boundary_z = NoFlux```

```neighbor_order = 1```

```random_seed = None```

```lattice_type = Cartesian```

```offset = 0``` 



In [ ]:
# Potts
from cc3d.core.PyCoreSpecs import Metadata, PottsCore


# Cell Type
Default ```Medium``` but can set new cell type

In [ ]:
# Cell Type
from cc3d.core.PyCoreSpecs import CellTypePlugin


# Initialization
This is like the very last one before the acutal simulation runs, parameters: 
- UniformInitializer
- BlobInitializer
- PIFInitializer